### Setup and imports
Let us first get our imports out of the way:

In [13]:
from pydrake.all import *

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from manipulation.meshcat_utils import AddMeshcatTriad

from pathlib import Path
import numpy as np

### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [66]:
abs_table_sdf_path = f"{Path.cwd()}/assets/table.sdf"
with open(abs_table_sdf_path, "r") as fin:
    table_sdf = fin.read()

scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{abs_table_sdf_path}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}

- add_model:
    name: ball0
    file: package://drake_models/manipulation_station/sphere.sdf
    default_free_body_pose:
        base_link:
            translation: [0, -0.5, 0.02]
            rotation: !Rpy {{ deg: [0, 0, 0] }}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

### Pick and Place Functions

Helper functions for designing grasp poses and trajectories.

In [71]:
def design_grasp_pose(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """Design a grasp pose 0.17m above the object."""
    p_OG = [0, 0, 0.1]
    R_OG = RotationMatrix.MakeXRotation(-np.pi/2)
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_OG, X_WG

def approach_pose(X_WG: RigidTransform) -> RigidTransform:
    """Create an approach pose 0.1m above the grasp pose."""
    X_WGApproach = RigidTransform(p=[0, 0, 0.1]) @ X_WG
    return X_WGApproach

def make_trajectory(
    X_Gs: list[RigidTransform], finger_values: np.ndarray, sample_times: list[float]
) -> tuple[Trajectory, PiecewisePolynomial]:
    """Convert keyframes to velocity trajectory and gripper commands."""
    piecewise = PiecewisePose.MakeLinear(sample_times, X_Gs)
    robot_velocity_trajectory = piecewise.MakeDerivative()
    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(sample_times, finger_values)
    return robot_velocity_trajectory, traj_wsg_command

### Jacobian-Based Velocity Controller

In [72]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],
            self._W,
            self._W
        )
        J_G = J_G[:, self.iiwa_start : self.iiwa_end + 1]
        v = np.linalg.pinv(J_G) @ V_G
        output.SetFromVector(v)

### Build Diagram with Pick and Place Controller

In [74]:
# Build the diagram with controller
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")
scenegraph = station.GetSubsystemByName("scene_graph")

# Clear previous meshcat objects
meshcat.Delete()

# Get initial poses
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))

# Get ball pose
ball_body = plant.GetBodyByName("base_link", plant.GetModelInstanceByName("ball0"))
X_WO_ball = plant.EvalBodyPoseInWorld(temp_plant_context, ball_body)

# Design grasp and approach poses
X_OG, X_WG_pick = design_grasp_pose(X_WO_ball)
X_WG_prepick = approach_pose(X_WG_pick)

# Visualize pick and prepick poses with RGB triads
AddMeshcatTriad(meshcat, "prepick_pose", length=0.15, radius=0.005, X_PT=X_WG_prepick)
AddMeshcatTriad(meshcat, "pick_pose", length=0.15, radius=0.005, X_PT=X_WG_pick)

# Add frame triads for gripper and ball bodies
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName("body"),
    length=0.15,
    radius=0.005
)
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=ball_body,
    length=0.1,
    radius=0.003
)

# Gripper states
opened = 0.107
closed = 0.0

In [75]:
# Define keyframes: start -> prepick -> pick -> close gripper -> return
keyframes = [
    (X_WGinitial, opened),
    (X_WG_prepick, opened),
    (X_WG_pick, opened),
    (X_WG_pick, closed),
    (X_WGinitial, closed),
]

# Build trajectory
gripper_poses = [kf[0] for kf in keyframes]
finger_states = np.array([kf[1] for kf in keyframes]).reshape(1, -1)
sample_times = [2 * i for i in range(len(gripper_poses))]
traj_V_G, traj_wsg_command = make_trajectory(gripper_poses, finger_states, sample_times)

In [76]:
# Add systems to diagram
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
controller = builder.AddSystem(PseudoInverseController(plant))
integrator = builder.AddSystem(Integrator(7))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connect systems
builder.Connect(V_G_source.get_output_port(), controller.V_G_port)
builder.Connect(controller.get_output_port(), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.q_port)
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()

# Run simulation
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant.GetMyContextFromRoot(context),
        plant.GetModelInstanceByName("iiwa"),
    ),
)
diagram.ForcedPublish(context)

print(f"Running simulation for {traj_V_G.end_time()} seconds")
meshcat.StartRecording()
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(traj_V_G.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

Running simulation for 8.0 seconds
